In [1]:
import plotly.graph_objects as go
import os
import pandas as pd

How many unique video id we crawl

In [16]:

df = pd.read_csv('/home/data/chensun/affi_project/purl_test/OpenWPM_2/US_youtube_trending_data.csv')
# df_top_1000 = df[:1000]
print("Total length of US trending videos: ", len(df))
df_subset = df.loc[:270000, ['video_id']]
print("We crawled: ", len(df_subset))

df_subset = df_subset.drop_duplicates(subset=['video_id'])
print("After deduplicate, we crawl ", len(df_subset), " unique trending video")

Total length of US trending videos:  264587
We crawled:  264587
After deduplicate, we crawl  46629  unique trending video


Excludes youtube video if 
(a) no urls in the description 
(b) all the urls in the description are from common social media platforms.

In [3]:
# extract from openwpm_2 "Alexa_top_sites.ipynb"
df_filtered = pd.read_csv('/home/data/chensun/affi_project/purl_test/OpenWPM_2/US_youtube_trending_dedup_removed.csv')
print(len(df_filtered))

38076


Excluded in Phase 0 ==> exclude links using url domain

In [2]:
record_path = '/home/data/chensun/affi_project/yt_records'

total_ignored_link = 0
for crawl_id in os.listdir(record_path):
    if crawl_id == 'dedup_links.ipynb' or crawl_id == 'yt_120k_v2':
        continue
    each_crawl = os.path.join(record_path, crawl_id)

    ignored_path = os.path.join(each_crawl, 'youtube_ignored_links.csv')
    df = pd.read_csv(ignored_path)
    
    total_ignored_link += df['excluded links'].sum()


print("total_link Excluded in Phase 0: ", total_ignored_link)

total_link Excluded in Phase 0:  449803


Affiliate by Regex 
== Count total links be lable as affiliate from the rule based classifier 
== rule_based_aff_yt

In [3]:
aff_folder = '/home/data/chensun/affi_project/purl/output/rule_based_aff_yt'
total_aff_link = 0
for crawl_id in os.listdir(aff_folder):
    
      
    each_crawl = os.path.join(aff_folder, crawl_id)

    aff_file_path = os.path.join(each_crawl, 'records.csv')
    df = pd.read_csv(aff_file_path)
    
    total_aff_link += len(df)

print("total_aff_link: ", total_aff_link)


total_aff_link:  17433


Processed after Phase 1 
== Count total links be lable as "others" from the rule based classifier 
== rule_based_others_yt

In [4]:
others_folder = '/home/data/chensun/affi_project/purl/output/rule_based_others_yt'
total_others_link = 0
for crawl_id in os.listdir(others_folder):
    
      
    each_crawl = os.path.join(others_folder, crawl_id)

    others_file_path = os.path.join(each_crawl, 'records.csv')
    df = pd.read_csv(others_file_path)
    
    total_others_link += len(df)

print("total_others_link: ", total_others_link)

total_others_link:  252269


Processed after Phase 0  
==  rule_based_aff_yt + rule_based_others_yt  

In [5]:
total_links = total_aff_link + total_others_link
print("total links processed after Phase 0", total_links)

total links processed after Phase 0 269702


count the unknown type links

In [6]:
others_folder = '/home/data/chensun/affi_project/purl/output/rule_based_others_yt'
total_unknown_links = 0
for crawl_id in os.listdir(others_folder):
    
      
    each_crawl = os.path.join(others_folder, crawl_id)

    others_file_path = os.path.join(each_crawl, 'rule_based_label.csv')
    df = pd.read_csv(others_file_path)
    unknown_label_df = df[df['final_rules_based_label'] == 'unknown']
    unique_unknown_visits = unknown_label_df.drop_duplicates(subset='visit_id')
    total_unknown_links += len(unique_unknown_visits)
    

print("total_unknown_link: ", total_unknown_links)

total_unknown_link:  0


In [7]:
#how many social media links

total_clicked_socialmedia_count = 0
others_folder = '/home/data/chensun/affi_project/purl/output/rule_based_others_yt'
# Define a list of domains to exclude
excluded_domains = ["facebook.com", "instagram.com", "twitter.com", "youtube.com", 
                    "linkedin.com", "pinterest.com", "weibo.com"]


for crawl_id in os.listdir(others_folder):
    
      
    each_crawl = os.path.join(others_folder, crawl_id)

    records_file_path = os.path.join(each_crawl, 'records.csv')
    df = pd.read_csv(records_file_path)
   
    is_excluded = df['landing_page_domain'].isin(excluded_domains)
    excluded_count = is_excluded.sum()
    total_clicked_socialmedia_count += excluded_count
   
print("Total Excluded Social Media: ", total_clicked_socialmedia_count)

Total Excluded Social Media:  141307


In [9]:
# Count links for ML

result  = '/home/data/chensun/affi_project/purl/output/results/04_25_yt_threshold=0.5_undersampling_5fold_CV_reducedF/phase1/0/all_others_data_with_model_0.5/classify_all_others.csv'
df = pd.read_csv(result)
df_aff = df[df['clabel'] == 'affiliate']
df_non_aff = df[df['clabel'] == 'others']
print("Affiliate by ML: ", len(df_aff))
print("Other by ML: ", len(df_non_aff))

Affiliate by ML:  4691
Other by ML:  57818


In [5]:

# Data for the Sankey diagram
labels = ["Total Links", "Excluded Social Media in Phase 0", "Phase 1", 
          "Labelled Aff by Regex", "Processed after Phase 1", "Excluded Social Media", 
          "Links for ML", "Duplicate links Dropped", "Apply the ML", "Labelled Aff by ML", "Lablled Other by ML"]

source = [0, 0, 2, 2, 4, 4, 6, 6, 8, 8]   # Indices correspond to 'labels'
target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
value = [214451, 192136, 9855, 182281, 111531, 70750 , 8241, 62509, 4691, 57818]

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
      pad=15,
      thickness=20,
      line=dict(color="black", width=0.5),
      label=labels,
      color="blue"
    ),
    link=dict(
      source=source,  # indices correspond to labels, eg source 0 -> target 1
      target=target,
      value=value,
      label=value  # display the values on the links
    ))])

fig.update_layout(title_text="Sankey Diagram for YouTube Affiliate Link Analysis", font_size=10)
fig.show()
